In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from sklearn.utils import shuffle
import pandas as pd 
from skimage.io import imread, imshow
from skimage.transform import rescale
from os import listdir
from sklearn.model_selection import train_test_split as tts
from skimage.color import rgb2hsv
from skimage.filters import threshold_otsu
from os import listdir
import pickle

In [2]:
 tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
with open('otsudataX','rb') as f:
    X = pickle.load(f)

with open('otsudataY','rb') as g:
    Y = pickle.load(g)


In [4]:
X.shape

(9957, 120, 160)

In [5]:
X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.33)

In [6]:
X_train.shape

(6671, 120, 160)

In [7]:
X_train = X_train.reshape(6671,120,160,1)
X_train.shape
# y_train[0:20]

(6671, 120, 160, 1)

In [8]:
y_train_cate = keras.utils.to_categorical(
    y_train,num_classes=4,dtype='float32'
)

y_train_cate[0:10]

array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [9]:
model = keras.Sequential([
    #cnn layers
    keras.layers.Conv2D(filters=32,kernel_size=(3, 3),activation='relu',input_shape=(120,160,1)),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(filters=32,kernel_size=(3, 3),activation='relu'),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(filters=32,kernel_size=(3, 3),activation='relu'),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(filters=64,kernel_size=(3, 3),activation='relu'),
    keras.layers.MaxPooling2D((2,2)),

    #dense normal ann
    keras.layers.Flatten(), 
    
    keras.layers.Dense(60,activation='relu'),
    keras.layers.Dense(30,activation='relu'),
    keras.layers.Dense(10,activation='relu'),
    
    keras.layers.Dense(4,activation='softmax'),
    #softmax normalises the probability of the output set
])

model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train,y_train_cate,epochs=20)

Epoch 1/20
209/209 [==============================] - 15s 28ms/step - loss: 1.3720 - accuracy: 0.2986
Epoch 2/20
209/209 [==============================] - 5s 26ms/step - loss: 1.0069 - accuracy: 0.5241
Epoch 3/20
209/209 [==============================] - 5s 26ms/step - loss: 0.7988 - accuracy: 0.6263
Epoch 4/20
209/209 [==============================] - 5s 26ms/step - loss: 0.7084 - accuracy: 0.6725
Epoch 5/20
209/209 [==============================] - 5s 26ms/step - loss: 0.6059 - accuracy: 0.7267
Epoch 6/20
209/209 [==============================] - 5s 26ms/step - loss: 0.5188 - accuracy: 0.7609
Epoch 7/20
209/209 [==============================] - 5s 26ms/step - loss: 0.4640 - accuracy: 0.7969
Epoch 8/20
209/209 [==============================] - 5s 26ms/step - loss: 0.3966 - accuracy: 0.8345
Epoch 9/20
209/209 [==============================] - 5s 26ms/step - loss: 0.3644 - accuracy: 0.8538
Epoch 10/20
209/209 [==============================] - 5s 26ms/step - loss: 0.2634 - accur

In [10]:
X_test = X_test.reshape(3286,120,160,1)
y_test_cate = keras.utils.to_categorical(
    y_test,num_classes=4,dtype='float32'
)

In [11]:
model.evaluate(X_test,y_test_cate)

103/103 [==============================] - 2s 14ms/step - loss: 1.9888 - accuracy: 0.7048


[1.9888017177581787, 0.7048082947731018]

In [12]:
yprob = model.predict(X_test)

In [13]:
yprobcl = [np.argmax(i) for i in yprob]

In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [17]:
accuracy_score(y_test,yprobcl)

0.7048082775410834

In [20]:
from sklearn.utils.multiclass import unique_labels
def plot(y_true,y_pred):    
    column = [f'Predicted {label}' for label in unique_labels(y_true)]
    indexx = [f'Actual {label}' for label in unique_labels(y_true)]
    table = pd.DataFrame(confusion_matrix(y_true,y_pred),columns=column,index=indexx)
    return table

plot(y_test,yprobcl)

,Predicted 0,Predicted 1,Predicted 2,Predicted 3
Actual 0,429,45,73,262
Actual 1,70,686,66,50
Actual 2,68,30,660,49
Actual 3,184,36,37,541
